# 04bis - Estimation bayésienne du paramètre xi

Dans ce notebook, nous allons essayer de mettre en place une estimation bayésienne pour le parmaètre xi à l'ensemble des stations du québec, après l'estimation normale.

In [25]:
using CSV, DataFrames, StatsBase, Random

using Distributions, Extremes, Optim

using Mamba, ForwardDiff, ProgressMeter #pour les MCMC

using IDF

import Plots

In [2]:
PROVINCES = ["NB", "NL", "NS", "ON", "PE", "QC"]#provinces considerees

DURATION = "24 h"

"24 h"

In [3]:
station_list = load_data("station_list")

filter!(row -> row.Province ∈ PROVINCES , station_list)#on ne selectionne que les stations qui nous interessent

first(station_list, 10)

,Name,Province,ID,Lat,Lon,Elevation
,String,String,String,Float64,Float64,Int64
1,BEECHWOOD,NB,8100512,46.53,-67.67,91
2,BELLEDUNE,NB,8100514,47.9,-65.83,7
3,BOUCTOUCHE CDA CS,NB,8100593,46.43,-64.77,35
4,CHARLO AUTO,NB,8100885,47.98,-66.33,42
5,MIRAMICHI RCS,NB,8100989,47.02,-65.47,33
6,EDMUNDSTON,NB,8101303,47.42,-68.32,154
7,FREDERICTON A,NB,8101500,45.87,-66.53,20
8,FREDERICTON CDA CS,NB,8101605,45.92,-66.62,35
9,MONCTON INTL A,NB,8103201,46.12,-64.68,70


Comme l'estimation classique n'a pas fonctionné à cause du nombre important de paramètres, nous allons donc procéder à une estimation bayésienne, avec comme paramètres initiaux ceux obtenus par une estimation de maximum de vraisemblance pour une loi de Gumbel (c.a.d. avec $\xi = 0$).

Commençons donc par construire ce tableau de valeurs initiales.

In [19]:
# same function as in IDF package, built it before integrating to IDF.jl
function solve_gumbel(y::Vector{Float64})
    μ₀ = mean(y)
    ϕ₀ = log(std(y))
    p₀ = [μ₀, ϕ₀]
    
    # la fonction objectif, comme on maximise la log_vraisemblance
    # il faut prendre l'oppose car optimize.jl minimise.
    function fobj(p::Vector{Float64})
        return -logL(y, p[1], exp(p[2]), 0.0)
    end
    
    
    p = p₀
    
    try
        res = optimize(fobj, p₀)
        
         
        if Optim.converged(res)
            p = Optim.minimizer(res)
        else
            @warn "The maximum likelihood algorithm did not find a solution. Maybe try with different initial values or with another method. The returned values are the initial values."
            p = p₀
        end
        
    catch
        println("Error of execution")
    end
   
    return p
end
    
μ₀ = []
ϕ₀ = []

for i in 1:(nrow(station_list))
    y = pcp(station_list[i, :ID], DURATION)
    
    p = solve_gumbel(y)
    
    append!(μ₀, p[1])
    append!(ϕ₀, p[2])
end

station_list[:μ₀] = Float64.(μ₀)
station_list[:ϕ₀] = Float64.(ϕ₀)

first(station_list, 10)

,Name,Province,ID,Lat,Lon,Elevation,μ₀,ϕ₀
,String,String,String,Float64,Float64,Int64,Float64,Float64
1,BEECHWOOD,NB,8100512,46.53,-67.67,91,55.1486,2.70978
2,BELLEDUNE,NB,8100514,47.9,-65.83,7,40.2054,2.39197
3,BOUCTOUCHE CDA CS,NB,8100593,46.43,-64.77,35,53.4377,2.57062
4,CHARLO AUTO,NB,8100885,47.98,-66.33,42,48.4714,2.66403
5,MIRAMICHI RCS,NB,8100989,47.02,-65.47,33,49.9964,2.78772
6,EDMUNDSTON,NB,8101303,47.42,-68.32,154,48.2441,2.79296
7,FREDERICTON A,NB,8101500,45.87,-66.53,20,50.8524,2.61132
8,FREDERICTON CDA CS,NB,8101605,45.92,-66.62,35,55.2421,2.73373
9,MONCTON INTL A,NB,8103201,46.12,-64.68,70,51.9352,2.87873


In [22]:
θ₀ = Float64.([station_list[:μ₀]; station_list[:ϕ₀]; 0.0])

673-element Vector{Float64}:
 55.14855306528422
 40.20542553881306
 53.43774890879652
 48.47140573289281
 49.996392488014884
 48.24412232109405
 50.852392666251404
 55.242107889525414
 51.93524893967279
 49.83618162207249
 51.209661078586855
 62.756030020231165
 67.97254136984631
  ⋮
  2.387908247372233
  2.1043510431058565
  2.287271290366869
  2.2640535479064514
  2.273393761089009
  2.0390156967528696
  2.010442168292835
  2.377784759339927
  2.131497963704006
  1.6611828658787275
  2.3630729736686393
  0.0

Maintenant que nous avons les valeurs initiales, lançons l'estimation bayésienne.

In [32]:
function xi_bayes(stations_df::DataFrame, niter::Int=5000, warmup::Int=2000)
    
    # ---- valeurs initiales
    N = nrow(stations_df)
    initialvalues = Float64.([stations_df[:μ₀]; stations_df[:ϕ₀]; 0.0])
    
    
    # ---- definition des fonctions qui serviront à la construction de la chaîne
    
    # la fonction de log vraisemblance est celle vue au notebook 4
    function logf(θ::Vector{Float64})
        μ = θ[1:N]
        ϕ = θ[N+1:(2*N)]
        ξ = θ[2*N+1]

        s=0
        for i in 1:N
            id = stations_df[i, :ID]
            s -= logL(pcp(id, DURATION), μ[i], exp(ϕ[i]), ξ) # on met l'opposé de la log-vraisemblance
        end

        return s
    end
    
    Δlogf(θ::Vector{Float64}) = ForwardDiff.gradient(logf, θ)
    
    function logfgrad(θ::Vector{Float64})
        ll = logf(θ)
        g = Δlogf(θ)
        return ll, g
    end
    
    # ---- lancement de la simulation par MCMC
    
    sim = Chains(niter, 2*N+1, start = (warmup + 1))
    θ = NUTSVariate(initialvalues, logfgrad)
    @showprogress for i in 1:niter
        sample!(θ, adapt = (i <= warmup))
        if i > warmup
            sim[i, :, 1] = θ
        end
    end
    
    # ---- renvoi de la chaîne
    
    return sim
    
end

xi_bayes (generic function with 3 methods)

In [33]:
xi_bayes(station_list)

LoadError: MethodError: no method matching (::var"#logf#22"{DataFrame, Int64})(::Vector{ForwardDiff.Dual{ForwardDiff.Tag{var"#logf#22"{DataFrame, Int64}, Float64}, Float64, 12}})
[0mClosest candidates are:
[0m  (::var"#logf#22")([91m::Vector{Float64}[39m) at In[32]:11

Les fonctions du package Mamba.jl semblant chancelantes pour des estimations multivariées, nous allons concevoir nous même l'algorithme. Nous allons donc pour ceci établir un algorithme d'échantillonage de Gibbs.

In [ ]:
function gibbs(stations_df::DataFrame, δ::Int, niter::Int=5000, warmup::Int=2000)
    
    # ---- valeurs initiales
    N = nrow(stations_df)
    initialvalues = Float64.([stations_df[:μ₀]; stations_df[:ϕ₀]; 0.0])
    
    
    # ---- definition des fonctions qui serviront à la construction de la chaîne

    # la fonction de log vraisemblance est celle vue au notebook 4
    function logf(θ::Vector{Float64})
        μ = θ[1:N]
        ϕ = θ[N+1:(2*N)]
        ξ = θ[2*N+1]

        s=0
        for i in 1:N
            id = stations_df[i, :ID]
            s -= logL(pcp(id, DURATION), μ[i], exp(ϕ[i]), ξ) # on met l'opposé de la log-vraisemblance
        end

        return s
    end
    
    Δlogf(θ::Vector{Float64}) = ForwardDiff.gradient(logf, θ)
    
    function logfgrad(θ::Vector{Float64})
        ll = logf(θ)
        g = Δlogf(θ)
        return ll, g
    end
    
    # échantillonage
    
    sim = Chains(niter, 2*N+1, start = (warmup + 1))
    
end
